<a href="https://colab.research.google.com/github/angeruzzi/ETL_DadosPluviometricos/blob/main/ETL_DadosPluviometricos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script de ETL para importação de dados Pluviométricos

O script a seguir foi desenvolvimento para importação dos dados pluviométricos de estações de medição de todo o Brasil disponibilizados pelo [INMET](https://bdmep.inmet.gov.br/).

Na execução do processo foram registradas **1276 estações de todo o Brasil** e importados mais de **328 mil registros de medições** mensais de 1970 a 2020, e ao final foram gerados arquivos consolidados por Unidade Federal.



O processo consistiu em:
1. Importar via acesso a APIs do INMET a lista de estações de medições disponíveis, assim como seus dados de localização;
2. Extrair e importar os registros dos arquivos gerados previamente por solicitação na plataforma [BDMEP](https://bdmep.inmet.gov.br/);
3. Consolidar as informações de todas as estações por estado e exportar em arquivos CSV para posterior utilização em análises.

Requisitos e instruções para executar o processo:
* Ambiente para execução do script em Python;
* Banco de dados MySQL com um database 'precipitacao' criado;
* Solicitar os arquivos das estações de medição na plataforma [BDMEP](https://bdmep.inmet.gov.br/) consolidados por mês; Antecipando que são disponibilizados arquivos de 2 tipos diferentes de estação (Automática e Convencional) e que devem ser mantidos separados por terem layouts distintos; 
</br>
</br>

Faço as seguintes ressalvas e considerações sobre o processo: 
* Foi assumido como premissa que cada etapa do processo será executada uma única vez, não havendo portanto tratativas para evitar a reexecução das etapas causando duplicações de informações caso isso ocorrer;
* O script foi desenvolvimento para execução local, não estando adaptado ou testado para execução em plataformas em nuvem como o Google Colab;
* Disponibilizei os arquivos que utilizei no processamento [aqui](https://github.com/angeruzzi/Datasource/tree/main/pluviometricos), mas no caso de solicitação dos arquivos no BDMEP deve ser mantido os mesmos parâmetros de solicitação original utilizados para evitar discrepância nos layouts, sendo eles: Tipo de Pontuação= Ponto; Tipo de Dados= Dados Mensais; Tipo de Estação= Ambas (solicitadas separadamente); Abrangência= País; Variáveis= Todas; Estações= Todas.

* O processo foi estabelecido para trabalhar com os dados consolidados mensalmente das estações de medição, porém os registros diários podem ser solicitados e o script adaptado para trabalhar com eles;
* Foram importadas outras informações de medição como dados de vento, temperatura e nebulosidade que podem ser utilizadas também.


E como observação final, ressalto que o desenvolvimento desse script foi com **intuito didático no uso de Python em um processo de ETL** com: Acesso a APIs, manipulação de diretórios e arquivos, acesso a base de dados e exportação de informações. Há softwares especializados na importação de registros de Pluviometria e diversas técnicas para consolidação das informações, assim 
como uma série de cuidados que devem ser tomados quanto a confiança desses dados devido principalmente a falha de equipamentos.    

Disponibilizei os arquivos finais consolidados por estado [aqui](https://github.com/angeruzzi/Datasource/blob/main/pluviometricos/DadosPluviometricosMensalPorEstado_1970a2020.rar), fazendo a ressalva que estes dados precisam de um melhor tratamento para qualquer uso com fins de análise.



---------------------------------
**Autor**: [Alessandro S. Angeruzzi](https://www.linkedin.com/in/alessandroangeruzzi/)

**License**: Creative Commons

---------------------------------


#Importação das Bibliotecas

In [ ]:
import mysql.connector
import requests
import json
import csv

from mysql.connector import errorcode
from datetime import datetime
from os import walk

#Configurações

In [ ]:
#Dados de Conexão com o banco de dados
db_connection = mysql.connector.connect(host='localhost', user='root', password='', database='precipitacao')

#Urls das APIs que serão consultadas
urls = [
    'https://apibdmep.inmet.gov.br/T/R/CO',
    'https://apibdmep.inmet.gov.br/T/R/NO',
    'https://apibdmep.inmet.gov.br/T/R/N',
    'https://apibdmep.inmet.gov.br/T/R/SU',
    'https://apibdmep.inmet.gov.br/T/R/S',
    'https://apibdmep.inmet.gov.br/M/R/CO',
    'https://apibdmep.inmet.gov.br/M/R/NO',
    'https://apibdmep.inmet.gov.br/M/R/N',
    'https://apibdmep.inmet.gov.br/M/R/SU',
    'https://apibdmep.inmet.gov.br/M/R/S'
]

#Endereço locais dos arquivos das estações previamente exportados
#Estações Automáticas
pathAutomaticas = '../Datasource/pluviometricos/automaticas'
#Estações Convencionais
pathConvencionais = '../Datasource/pluviometricos/convencionais'

#Endereço dos arquivos finais por estado que serão gerados ao final do script
pathConvertidos = '../Datasource/pluviometricos/convertidos'

#Funções de tratamento dos dados

In [ ]:
def trata_str(s):
    s_ret = "NULL"
    if s:
        s_ret = "'"+s+"'"
    return s_ret

def trata_num(s):
    s_ret = "NULL"
    if s and (s != 'null'):
        s_ret = str(s)
    return s_ret

def trata_dt(s):
    s_ret = "NULL"
    if s:
        s_ret = "'"+s[:10]+"'"
    return s_ret

def trata_dt_split(s):
    s_ret_dt  = "NULL"
    s_ret_mes = "NULL"
    s_ret_ano = "NULL"
    if s:
        a_dt = s.split('-')
        s_ret_dt  = "'"+s+"'"
        s_ret_mes = str(a_dt[1])
        s_ret_ano = str(a_dt[0])
    return s_ret_dt, s_ret_mes, s_ret_ano

#Geração das tabelas no Banco de Dados

In [ ]:
cursor = db_connection.cursor()

cursor.execute("CREATE TABLE `estacoes` ( `CD_ESTACAO` VARCHAR(10) NOT NULL , `DC_NOME` VARCHAR(50) NULL , `TP_ESTACAO` VARCHAR(15) NULL , `CD_SITUACAO` VARCHAR(15) NULL , `DT_INICIO_OPERACAO` DATE NULL , `DT_FIM_OPERACAO` DATE NULL , `SG_REGIAO` CHAR(3) NULL , `CD_DISTRITO` INT NULL , `SG_ESTADO` CHAR(2) NULL , `VL_LATITUDE` DECIMAL(14,4) NULL DEFAULT NULL , `VL_LONGITUDE` DECIMAL(14,4) NULL DEFAULT NULL , `VL_ALTITUDE` DECIMAL(14,4) NULL DEFAULT NULL , `SG_ENTIDADE` VARCHAR(20) NULL , `CD_WSI` VARCHAR(30) NULL , `CD_OSCAR` VARCHAR(30) NULL , `FL_CAPITAL` VARCHAR(5) NULL , PRIMARY KEY (`CD_ESTACAO`)) ENGINE = InnoDB;")

cursor.execute("CREATE TABLE `arquivos` ( `ID` INT NOT NULL AUTO_INCREMENT , `NOME` VARCHAR(50) NULL , `CD_ESTACAO` VARCHAR(10) NULL , `DT_INI` DATE NULL , `DT_FIM` DATE NULL , `PERIODICIDADE` VARCHAR(10) NULL , `DT_IMPORT` DATETIME NULL , PRIMARY KEY (`ID`)) ENGINE = InnoDB;")

cursor.execute("CREATE TABLE `registros` ( `ID` BIGINT NOT NULL AUTO_INCREMENT , `ID_ARQUIVO` INT NULL , `CD_ESTACAO` VARCHAR(10) NULL , `PERIODICIDADE` VARCHAR(10) NULL , `DT_CADASTRO` DATETIME NULL DEFAULT CURRENT_TIMESTAMP, `DT_MEDICAO` DATE NULL , `ANO` INT NULL, `MES` INT NULL,`DIRECAO_VENTO` INT NULL , `EVAPORACAO_PICHE` DECIMAL(14,4) NULL DEFAULT NULL , `EVAPOTRANSPIRACAO_POTENCIAL` DECIMAL(14,4) NULL DEFAULT NULL , `EVAPOTRANSPIRACAO_REAL` DECIMAL(14,4) NULL DEFAULT NULL , `INSOLACAO_TOTAL` DECIMAL(14,4) NULL DEFAULT NULL , `NEBULOSIDADE_MEDIA` DECIMAL(14,4) NULL DEFAULT NULL , `NUMERO_DIAS_PRECIP` INT NULL , `PRECIPITACAO_TOTAL` DECIMAL(14,4) NULL DEFAULT NULL , `PRESSAO_ATMOSFERICA` DECIMAL(14,4) NULL DEFAULT NULL , `TEMPERATURA_MAXIMA_MEDIA` DECIMAL(14,4) NULL DEFAULT NULL , `TEMPERATURA_MEDIA_COMPENSADA` DECIMAL(14,4) NULL DEFAULT NULL , `TEMPERATURA_MINIMA_MEDIA` DECIMAL(14,4) NULL DEFAULT NULL , `UMID_RELATIVA_AR_MEDIA` DECIMAL(14,4) NULL DEFAULT NULL , `VENTO_VELOC_MAXIMA` DECIMAL(14,4) NULL DEFAULT NULL , `VENTO_VELOC_MEDIA` DECIMAL(14,4) NULL DEFAULT NULL , `VISIBILIDADE_MEDIA` DECIMAL(14,4) NULL DEFAULT NULL , PRIMARY KEY (`ID`)) ENGINE = InnoDB;")

cursor.close()
db_connection.commit()

#Consulta e Importação dos dados das Estações de Medição

In [ ]:
#Configuração das Requisições
payload={}
headers = {
'User-Agent': 'Projeto ETL Pluviometrico',
'Content-Type': 'application/json; charset=utf-8',
}

cursor = db_connection.cursor()

for url in urls:
    req = requests.request("GET", url, headers=headers, data=payload)
    if req.status_code == 200:
        req_j = json.loads(req.content)
        for item in req_j:

            item['SG_REGIAO']   = trata_str(item['SG_REGIAO'])
            item['CD_OSCAR']    = trata_str(item['CD_OSCAR'])
            item['DC_NOME']     = trata_str(item['DC_NOME'])
            item['FL_CAPITAL']  = trata_str(item['FL_CAPITAL'])
            item['CD_SITUACAO'] = trata_str(item['CD_SITUACAO'])
            item['TP_ESTACAO']  = trata_str(item['TP_ESTACAO'])
            item['VL_LATITUDE'] = trata_num(item['VL_LATITUDE'])
            item['CD_WSI']      = trata_str(item['CD_WSI'])
            item['CD_DISTRITO'] = trata_str(item['CD_DISTRITO'])
            item['VL_ALTITUDE'] = trata_num(item['VL_ALTITUDE'])
            item['SG_ESTADO']   = trata_str(item['SG_ESTADO'])
            item['SG_ENTIDADE'] = trata_str(item['SG_ENTIDADE'])
            item['CD_ESTACAO']  = trata_str(item['CD_ESTACAO'])
            item['VL_LONGITUDE']= trata_num(item['VL_LONGITUDE'])
            item['DT_INICIO_OPERACAO'] = trata_dt(item['DT_INICIO_OPERACAO'])
            item['DT_FIM_OPERACAO']    = trata_dt(item['DT_FIM_OPERACAO'])

            SQL = "INSERT INTO `estacoes`(`CD_ESTACAO`, `DC_NOME`, `TP_ESTACAO`, `CD_SITUACAO`, `DT_INICIO_OPERACAO`, `DT_FIM_OPERACAO`, `SG_REGIAO`, `CD_DISTRITO`, `SG_ESTADO`, `VL_LATITUDE`, `VL_LONGITUDE`, `VL_ALTITUDE`, `SG_ENTIDADE`, `CD_WSI`, `CD_OSCAR`, `FL_CAPITAL`) " 
            SQL = SQL+"VALUES ("+item['CD_ESTACAO']+","+item['DC_NOME']+", \
                               "+item['TP_ESTACAO']+","+item['CD_SITUACAO']+", \
                               "+item['DT_INICIO_OPERACAO']+", "+item['DT_FIM_OPERACAO'] +", \
                               "+item['SG_REGIAO']+", "+item['CD_DISTRITO']+", \
                               "+item['SG_ESTADO']+", "+item['VL_LATITUDE']+", \
                               "+item['VL_LONGITUDE']+", "+item['VL_ALTITUDE']+", \
                               "+item['SG_ENTIDADE']+", "+item['CD_WSI']+", \
                               "+item['CD_OSCAR']+", "+item['FL_CAPITAL']+")"

            cursor.execute(SQL)
    else:
        print('ERRO NA REQUISIÇÃO')
        print('URL:'+url)
        print('status_code:'+req.status_code)
        break

cursor.close()
db_connection.commit()

#Importação dos dados das Estações Automáticas

In [ ]:
files = []

#Percorrendo o diretório com os arquivos
for (dirpath, dirnames, filenames) in walk(pathAutomaticas):
    files.extend(filenames)
    break

cursor = db_connection.cursor()

#Processando cada arqquivo
for arquivo in files:
    name_file = arquivo
    path_file = pathAutomaticas + '/' + name_file

    print('Importando Arquivo:', name_file)
    with open(path_file, 'r', encoding='utf-8') as csv_file:
        linhas = csv_file.read()
        linhas = linhas.split('\n')

        CD_ESTACAO    = linhas[1].split(':')[1].lstrip()
        DT_INI        = linhas[6].split(':')[1].lstrip()
        DT_FIM        = linhas[7].split(':')[1].lstrip()
        PERIODICIDADE = linhas[8].split(':')[1].lstrip()   
    
        #Cadastro do Arquivo
        SQL_A = "INSERT INTO `arquivos`(`NOME`, `CD_ESTACAO`, `DT_INI`, `DT_FIM`, `PERIODICIDADE`, `DT_IMPORT`)" 
        SQL_A = SQL_A+" VALUES ('"+name_file+"','"+CD_ESTACAO+"','"+DT_INI+"', '"+DT_FIM+"', '"+PERIODICIDADE+"', NOW())"

        cursor.execute(SQL_A)
        
        #Recupera a chave do arquivo inserido
        cursor.execute("SELECT max(`ID`) as id FROM `arquivos`")
        result = cursor.fetchall()
        ID_ARQUIVO = "NULL"
        for id in result:
            ID_ARQUIVO = id[0]
        
        #Percorre as Medições do arquivo e Insere na Base
        for i in range(11,len(linhas),1):
            linha = linhas[i].strip()
            if linha != "":
                campos = linha.split(';')

                DT_MEDICAO, MES, ANO = trata_dt_split(campos[0])
                NUMERO_DIAS_PRECIP   = trata_num(campos[1])
                PRECIPITACAO_TOTAL   = trata_num(campos[2])
                PRESSAO_ATMOSFERICA  = trata_num(campos[3])
                TEMPERATURA_MEDIA_COMPENSADA = trata_num(campos[4])
                VENTO_VELOC_MAXIMA   = trata_num(campos[5])
                VENTO_VELOC_MEDIA    = trata_num(campos[6])

                SQL_reg = "INSERT INTO `registros`(`ID_ARQUIVO`, `CD_ESTACAO`, `PERIODICIDADE`, `DT_CADASTRO`, `DT_MEDICAO`, `ANO`, `MES`, `NUMERO_DIAS_PRECIP`, `PRECIPITACAO_TOTAL`, `PRESSAO_ATMOSFERICA`, `TEMPERATURA_MEDIA_COMPENSADA`, `VENTO_VELOC_MAXIMA`, `VENTO_VELOC_MEDIA`)"
                SQL_reg = SQL_reg+" VALUES ("+str(ID_ARQUIVO)+", \
                '"+CD_ESTACAO+"', '"+PERIODICIDADE+"', NOW(), \
                "+DT_MEDICAO+", "+ANO+", "+MES+", "+NUMERO_DIAS_PRECIP+",\
                "+PRECIPITACAO_TOTAL+", "+PRESSAO_ATMOSFERICA+", \
                "+TEMPERATURA_MEDIA_COMPENSADA+", "+VENTO_VELOC_MAXIMA+", \
                "+VENTO_VELOC_MEDIA+")"

                cursor.execute(SQL_reg)

print("Fim da Importação")

cursor.close()
db_connection.commit()

Importando Arquivo: dados_A001_M_2000-05-06_2020-12-31.csv
Importando Arquivo: dados_A002_M_2001-05-28_2020-12-31.csv
Importando Arquivo: dados_A003_M_2001-05-24_2020-12-31.csv
Importando Arquivo: dados_A004_M_2001-05-30_2018-06-20.csv
Importando Arquivo: dados_A005_M_2001-06-03_2020-12-31.csv
Importando Arquivo: dados_A006_M_2001-06-07_2006-09-24.csv
Importando Arquivo: dados_A007_M_2003-02-19_2005-08-15.csv
Importando Arquivo: dados_A008_M_2003-06-12_2013-02-19.csv
Importando Arquivo: dados_A009_M_2004-12-16_2020-12-31.csv
Importando Arquivo: dados_A010_M_2005-03-03_2020-12-31.csv
Importando Arquivo: dados_A011_M_2006-07-14_2020-12-31.csv
Importando Arquivo: dados_A012_M_2006-10-20_2020-12-31.csv
Importando Arquivo: dados_A013_M_2007-07-12_2020-12-31.csv
Importando Arquivo: dados_A014_M_2007-07-17_2020-12-31.csv
Importando Arquivo: dados_A015_M_2007-02-02_2020-12-31.csv
Importando Arquivo: dados_A016_M_2007-05-22_2020-12-31.csv
Importando Arquivo: dados_A017_M_2007-04-17_2020-12-31.c

Importando Arquivo: dados_A302_M_2003-11-01_2020-12-31.csv
Importando Arquivo: dados_A303_M_2003-02-24_2020-12-31.csv
Importando Arquivo: dados_A304_M_2003-02-23_2020-12-31.csv
Importando Arquivo: dados_A305_M_2003-02-17_2020-12-31.csv
Importando Arquivo: dados_A306_M_2003-02-11_2020-12-31.csv
Importando Arquivo: dados_A307_M_2003-02-20_2020-12-31.csv
Importando Arquivo: dados_A308_M_2003-02-05_2020-12-31.csv
Importando Arquivo: dados_A309_M_2004-11-19_2020-12-31.csv
Importando Arquivo: dados_A310_M_2004-11-15_2020-12-31.csv
Importando Arquivo: dados_A311_M_2004-11-17_2020-12-31.csv
Importando Arquivo: dados_A312_M_2004-11-23_2020-12-31.csv
Importando Arquivo: dados_A313_M_2006-12-21_2020-12-31.csv
Importando Arquivo: dados_A314_M_2007-05-25_2020-12-31.csv
Importando Arquivo: dados_A315_M_2007-05-30_2020-12-31.csv
Importando Arquivo: dados_A316_M_2007-01-06_2020-12-31.csv
Importando Arquivo: dados_A317_M_2007-01-05_2020-12-31.csv
Importando Arquivo: dados_A318_M_2007-05-21_2020-12-31.c

Importando Arquivo: dados_A508_M_2002-12-14_2020-12-31.csv
Importando Arquivo: dados_A509_M_2004-12-18_2020-12-31.csv
Importando Arquivo: dados_A510_M_2005-09-14_2020-12-31.csv
Importando Arquivo: dados_A511_M_2006-02-21_2020-12-31.csv
Importando Arquivo: dados_A512_M_2006-05-11_2020-12-31.csv
Importando Arquivo: dados_A513_M_2006-07-27_2020-12-31.csv
Importando Arquivo: dados_A514_M_2006-06-08_2020-12-31.csv
Importando Arquivo: dados_A515_M_2006-07-12_2020-12-31.csv
Importando Arquivo: dados_A516_M_2006-07-15_2020-12-31.csv
Importando Arquivo: dados_A517_M_2006-08-29_2020-12-31.csv
Importando Arquivo: dados_A518_M_2007-05-25_2020-12-31.csv
Importando Arquivo: dados_A519_M_2006-07-14_2020-12-31.csv
Importando Arquivo: dados_A520_M_2006-07-17_2020-12-31.csv
Importando Arquivo: dados_A521_M_2006-10-09_2020-12-31.csv
Importando Arquivo: dados_A522_M_2006-08-20_2020-12-31.csv
Importando Arquivo: dados_A523_M_2006-08-21_2020-12-31.csv
Importando Arquivo: dados_A524_M_2006-08-16_2020-12-31.c

Importando Arquivo: dados_A738_M_2007-06-24_2020-12-31.csv
Importando Arquivo: dados_A739_M_2007-11-04_2020-12-31.csv
Importando Arquivo: dados_A740_M_2007-10-31_2020-12-31.csv
Importando Arquivo: dados_A741_M_2008-04-23_2020-12-31.csv
Importando Arquivo: dados_A742_M_2008-03-07_2020-12-31.csv
Importando Arquivo: dados_A743_M_2008-06-25_2020-12-31.csv
Importando Arquivo: dados_A744_M_2017-12-19_2020-12-31.csv
Importando Arquivo: dados_A745_M_2008-07-24_2017-05-17.csv
Importando Arquivo: dados_A746_M_2008-07-02_2020-12-31.csv
Importando Arquivo: dados_A747_M_2008-04-21_2020-12-31.csv
Importando Arquivo: dados_A748_M_2010-06-18_2020-12-31.csv
Importando Arquivo: dados_A749_M_2008-06-15_2020-12-31.csv
Importando Arquivo: dados_A750_M_2008-06-10_2020-12-31.csv
Importando Arquivo: dados_A751_M_2008-06-02_2020-12-31.csv
Importando Arquivo: dados_A752_M_2008-05-27_2020-12-31.csv
Importando Arquivo: dados_A753_M_2008-07-16_2020-12-31.csv
Importando Arquivo: dados_A754_M_2008-09-28_2020-12-31.c

Importando Arquivo: dados_A922_M_2006-11-30_2020-12-31.csv
Importando Arquivo: dados_A923_M_2017-10-25_2020-12-31.csv
Importando Arquivo: dados_A924_M_2007-05-22_2020-12-31.csv
Importando Arquivo: dados_A925_M_2007-07-10_2020-12-31.csv
Importando Arquivo: dados_A926_M_2008-04-08_2020-12-31.csv
Importando Arquivo: dados_A927_M_2008-02-28_2020-12-31.csv
Importando Arquivo: dados_A928_M_2008-04-15_2020-12-31.csv
Importando Arquivo: dados_A929_M_2008-04-12_2020-12-31.csv
Importando Arquivo: dados_A930_M_2008-06-03_2020-12-31.csv
Importando Arquivo: dados_A931_M_2008-05-29_2020-12-31.csv
Importando Arquivo: dados_A932_M_2008-01-23_2020-12-31.csv
Importando Arquivo: dados_A933_M_2008-08-13_2020-12-31.csv
Importando Arquivo: dados_A934_M_2008-01-28_2020-12-31.csv
Importando Arquivo: dados_A935_M_2008-05-29_2020-12-31.csv
Importando Arquivo: dados_A936_M_2008-01-23_2020-12-31.csv
Importando Arquivo: dados_A937_M_2008-01-29_2020-12-31.csv
Importando Arquivo: dados_A938_M_2008-08-25_2020-12-31.c

#Importação dos dados das Estações Convencionais

In [ ]:
files = []

for (dirpath, dirnames, filenames) in walk(pathConvencionais):
    files.extend(filenames)
    break

cursor = db_connection.cursor()

for arquivo in files:
    name_file = arquivo
    path_file = pathConvencionais + '/' + name_file

    print('Importando Arquivo:', name_file)
    with open(path_file, 'r', encoding='utf-8') as csv_file:
        linhas = csv_file.read()
        linhas = linhas.split('\n')

        CD_ESTACAO    = linhas[1].split(':')[1].lstrip()
        DT_INI        = linhas[6].split(':')[1].lstrip()
        DT_FIM        = linhas[7].split(':')[1].lstrip()
        PERIODICIDADE = linhas[8].split(':')[1].lstrip()   
    
        #Cadastro do Arquivo
        SQL_A = "INSERT INTO `arquivos`(`NOME`, `CD_ESTACAO`, `DT_INI`, `DT_FIM`, `PERIODICIDADE`, `DT_IMPORT`)" 
        SQL_A = SQL_A+" VALUES ('"+name_file+"','"+CD_ESTACAO+"','"+DT_INI+"', '"+DT_FIM+"', '"+PERIODICIDADE+"', NOW())"

        cursor.execute(SQL_A)
        
        #Recupera a chave do arquivo inserido
        cursor.execute("SELECT max(`ID`) as id FROM `arquivos`")
        result = cursor.fetchall()
        ID_ARQUIVO = "NULL"
        for id in result:
            ID_ARQUIVO = id[0]
        
        #Percorre as Medições e Insere na Base
        for i in range(11,len(linhas),1):
            linha = linhas[i].strip()
            if linha != "":
                campos = linha.split(';')

                DT_MEDICAO, MES, ANO         = trata_dt_split(campos[0])
                DIRECAO_VENTO                = trata_num(campos[1])
                EVAPORACAO_PICHE             = trata_num(campos[2])
                EVAPOTRANSPIRACAO_POTENCIAL  = trata_num(campos[3])
                EVAPOTRANSPIRACAO_REAL       = trata_num(campos[4])
                INSOLACAO_TOTAL              = trata_num(campos[5])
                NEBULOSIDADE_MEDIA           = trata_num(campos[6])
                NUMERO_DIAS_PRECIP           = trata_num(campos[7])
                PRECIPITACAO_TOTAL           = trata_num(campos[8])
                PRESSAO_ATMOSFERICA          = trata_num(campos[9])
                TEMPERATURA_MAXIMA_MEDIA     = trata_num(campos[10])
                TEMPERATURA_MEDIA_COMPENSADA = trata_str(campos[11])
                TEMPERATURA_MINIMA_MEDIA     = trata_num(campos[12])
                UMID_RELATIVA_AR_MEDIA       = trata_num(campos[13])
                VENTO_VELOC_MAXIMA           = trata_num(campos[14])
                VENTO_VELOC_MEDIA            = trata_num(campos[15])
                VISIBILIDADE_MEDIA           = trata_num(campos[16])            

                SQL_reg = "INSERT INTO `registros`(`ID_ARQUIVO`, `CD_ESTACAO`, `PERIODICIDADE`, `DT_CADASTRO`, `DT_MEDICAO`, `ANO`, `MES`, `DIRECAO_VENTO`, `EVAPORACAO_PICHE`, `EVAPOTRANSPIRACAO_POTENCIAL`, `EVAPOTRANSPIRACAO_REAL`, `INSOLACAO_TOTAL`, `NEBULOSIDADE_MEDIA`, `NUMERO_DIAS_PRECIP`, `PRECIPITACAO_TOTAL`, `PRESSAO_ATMOSFERICA`, `TEMPERATURA_MAXIMA_MEDIA`, `TEMPERATURA_MEDIA_COMPENSADA`, `TEMPERATURA_MINIMA_MEDIA`, `UMID_RELATIVA_AR_MEDIA`, `VENTO_VELOC_MAXIMA`, `VENTO_VELOC_MEDIA`, `VISIBILIDADE_MEDIA`)"
                SQL_reg = SQL_reg+" VALUES("+str(ID_ARQUIVO)+", \
                '"+CD_ESTACAO+"', '"+PERIODICIDADE+"', NOW(), \
                "+DT_MEDICAO+", "+ANO+", "+MES+", "+DIRECAO_VENTO+", \
                "+EVAPORACAO_PICHE+", "+EVAPOTRANSPIRACAO_POTENCIAL+",\
                "+EVAPOTRANSPIRACAO_REAL+", "+INSOLACAO_TOTAL+", \
                "+NEBULOSIDADE_MEDIA+", "+NUMERO_DIAS_PRECIP+", \
                "+PRECIPITACAO_TOTAL+", "+PRESSAO_ATMOSFERICA+", \
                "+TEMPERATURA_MAXIMA_MEDIA+", "+TEMPERATURA_MEDIA_COMPENSADA+", \
                "+TEMPERATURA_MINIMA_MEDIA+", "+UMID_RELATIVA_AR_MEDIA+", \
                "+VENTO_VELOC_MAXIMA+", "+VENTO_VELOC_MEDIA+", \
                "+VISIBILIDADE_MEDIA+")"
                cursor.execute(SQL_reg)

print("Fim da Importação")
##Encerra a conexão com o banco
cursor.close()
db_connection.commit()

Importando Arquivo: dados_82014_M_1970-01-01_1977-05-09.csv
Importando Arquivo: dados_82024_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_82029_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_82042_M_1970-01-10_2020-12-31.csv
Importando Arquivo: dados_82047_M_1984-08-19_2020-12-31.csv
Importando Arquivo: dados_82067_M_1970-01-01_2020-01-30.csv
Importando Arquivo: dados_82092_M_1981-03-31_1995-08-30.csv
Importando Arquivo: dados_82093_M_1970-01-01_1981-01-30.csv
Importando Arquivo: dados_82095_M_1970-01-01_1993-01-30.csv
Importando Arquivo: dados_82098_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_82100_M_1970-01-01_1979-04-30.csv
Importando Arquivo: dados_82103_M_1970-01-01_1979-03-31.csv
Importando Arquivo: dados_82105_M_1970-01-01_1980-12-30.csv
Importando Arquivo: dados_82106_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_82108_M_1970-01-01_1983-09-30.csv
Importando Arquivo: dados_82113_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_82141_M_1970-0

Importando Arquivo: dados_82983_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_82985_M_1970-01-01_1990-12-30.csv
Importando Arquivo: dados_82986_M_1970-01-01_2015-04-30.csv
Importando Arquivo: dados_82987_M_1970-01-01_1975-09-10.csv
Importando Arquivo: dados_82988_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_82989_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_82990_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_82991_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_82992_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_82994_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_82995_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_82996_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_82998_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_83004_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_83007_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_83010_M_1970-01-01_1974-12-31.csv
Importando Arquivo: dados_83013_M_1976-1

Importando Arquivo: dados_83401_M_1985-05-09_1988-07-05.csv
Importando Arquivo: dados_83405_M_1970-01-01_2014-07-02.csv
Importando Arquivo: dados_83408_M_1970-01-01_2020-10-31.csv
Importando Arquivo: dados_83409_M_1976-06-30_1978-04-30.csv
Importando Arquivo: dados_83410_M_1991-12-31_2016-01-29.csv
Importando Arquivo: dados_83412_M_1970-01-01_1985-09-30.csv
Importando Arquivo: dados_83422_M_1986-03-31_1997-07-31.csv
Importando Arquivo: dados_83423_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_83427_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_83428_M_1976-04-06_2020-12-31.csv
Importando Arquivo: dados_83432_M_1976-04-07_1987-07-29.csv
Importando Arquivo: dados_83435_M_1976-02-29_1977-01-31.csv
Importando Arquivo: dados_83437_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_83438_M_1979-05-07_1997-10-09.csv
Importando Arquivo: dados_83440_M_1970-01-01_1986-12-31.csv
Importando Arquivo: dados_83441_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_83442_M_1970-0

Importando Arquivo: dados_83707_M_1970-01-01_1991-04-30.csv
Importando Arquivo: dados_83712_M_1970-01-01_1973-12-31.csv
Importando Arquivo: dados_83713_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_83714_M_1970-01-01_2019-03-19.csv
Importando Arquivo: dados_83716_M_1970-01-01_2012-05-31.csv
Importando Arquivo: dados_83718_M_1971-07-15_2020-12-31.csv
Importando Arquivo: dados_83719_M_1970-01-01_1987-10-31.csv
Importando Arquivo: dados_83723_M_1970-01-01_1991-04-30.csv
Importando Arquivo: dados_83726_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_83727_M_1970-01-01_1970-08-30.csv
Importando Arquivo: dados_83728_M_1970-01-01_1991-04-30.csv
Importando Arquivo: dados_83729_M_1970-01-01_1978-12-31.csv
Importando Arquivo: dados_83730_M_1970-01-01_1991-04-30.csv
Importando Arquivo: dados_83732_M_1970-01-01_1995-05-30.csv
Importando Arquivo: dados_83736_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_83737_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_83738_M_1970-0

Importando Arquivo: dados_83955_M_1976-09-30_1985-03-31.csv
Importando Arquivo: dados_83956_M_1970-01-01_1982-07-31.csv
Importando Arquivo: dados_83957_M_1970-01-01_1976-06-30.csv
Importando Arquivo: dados_83959_M_1970-01-01_1975-09-30.csv
Importando Arquivo: dados_83960_M_1970-01-01_1971-06-30.csv
Importando Arquivo: dados_83961_M_1984-08-31_2013-11-29.csv
Importando Arquivo: dados_83963_M_1970-01-01_1978-11-30.csv
Importando Arquivo: dados_83964_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_83966_M_1970-01-01_1974-11-20.csv
Importando Arquivo: dados_83967_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_83972_M_1998-10-03_2011-11-30.csv
Importando Arquivo: dados_83976_M_1970-01-01_1978-06-30.csv
Importando Arquivo: dados_83978_M_1970-01-01_2006-12-11.csv
Importando Arquivo: dados_83980_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_83983_M_1970-01-01_1972-11-30.csv
Importando Arquivo: dados_83985_M_1970-01-01_2020-12-31.csv
Importando Arquivo: dados_83994_M_1970-0

#Consolidação dos dados por Unidade Federal e Geração dos arquivos

In [ ]:
cursor = db_connection.cursor()

##Consulta lista de Estados disponíveis
cursor.execute("SELECT DISTINCT `SG_ESTADO` FROM `estacoes` ORDER BY `SG_ESTADO`")
estados = cursor.fetchall()
for linha in estados:
    uf = linha[0]
    print(uf)
    cursor.execute("SELECT `ANO`, `MES`, \
    (case when `NUM_MED` > 0 then SUM_PREC/NUM_MED else NULL end) AS PREC, \
    NUM_MED FROM \
    ( SELECT `ANO`, `MES`, SUM(`PRECIPITACAO_TOTAL`) AS SUM_PREC, \
    SUM(case when `PRECIPITACAO_TOTAL` IS NULL then 0 else 1 end) AS NUM_MED \
    FROM `registros` WHERE `CD_ESTACAO` IN(\
    SELECT `CD_ESTACAO` FROM `estacoes` WHERE `SG_ESTADO` = '"+uf+"') \
    GROUP BY `ANO`, `MES` ) t ORDER BY `ANO`, `MES`")
    registros = cursor.fetchall()

    name_file = 'PluvMensal_'+uf+'.csv'
    path_file = pathConvertidos + '/' + name_file
    
    with open(path_file, 'w', newline='', encoding='utf-8') as file:

        writer = csv.writer(file, delimiter=';')
        writer.writerow(["Periodo", "PrecipitacaoTotal"])

        for linha_reg in registros:
            ano  = str(linha_reg[0])
            mes  = str(linha_reg[1])
            prec = str(linha_reg[2])
            writer.writerow([ano+"-"+mes, prec])

print("Fim da Geração de Arquivos")
##Encerra a conexão com o banco
cursor.close()
db_connection.commit()

AC
AL
AM
AP
BA
CE
DF
ES
GO
MA
MG
MS
MT
PA
PB
PE
PI
PR
RJ
RN
RO
RR
RS
SC
SE
SP
TO
Fim da Geração de Arquivos


In [ ]:
db_connection.close()